In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install transformers datasets

In [3]:
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('punkt')

# from datasets import load_dataset
# import datasets
# from datasets import Dataset

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Get the Idioms

Source: It’s not Rocket Science: Interpreting Figurative Language in Narratives. Data from https://github.com/tuhinjubcse/FigurativeNarrativeBenchmark.

(train + dev + test everything)


In [ ]:
# base_path = '/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Data - Not Rocket Science'

# idioms_list = []
# for dirnames, dirs, files in os.walk(base_path):
#     for filename in files:
#         filename_without_extension, extension = os.path.splitext(filename)
#         if extension == '.jsonl':
#             df_new = pd.read_json(dirnames+"/"+filename, lines=True)
            
#             if 'idiom' in df_new.columns:
#                 idioms_list.append(df_new['idiom'])

In [ ]:
# flat_list = [item for sublist in idioms_list for item in sublist]
# idioms_list_1 = set(flat_list)
# print("Total number of idioms:", len(idioms_list_1))

Source: MWEs from the SemEval task 

(train + dev + test everything)

In [ ]:
base_path = "/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Data - SemEval 2022"

idioms_list = []
for dirnames, dirs, files in os.walk(base_path):
    for filename in files:
        filename_without_extension, extension = os.path.splitext(filename)
        if extension == '.csv':
            df_new = pd.read_csv(dirnames+"/"+filename)

            if 'MWE' in df_new.columns:
                idioms_list.append(df_new.loc[df_new['Language'] == 'EN', 'MWE'])

In [ ]:
flat_list = [item for sublist in idioms_list for item in sublist]
idioms_list_2 = set(flat_list)
print("Total number of idioms:", len(idioms_list_2))

Total number of idioms: 273


In [ ]:
# combine the idioms
idioms_list = []
idioms_list.extend(list(idioms_list_1))
idioms_list.extend(list(idioms_list_2))
len(idioms_list)

273

In [ ]:
idioms_list[:5]

['wet blanket',
 'health check',
 'pillow slip',
 'traffic control',
 'basket case']

## Get the data

### Data 1

Source: SemEval 2022 Task 2: Multilingual Idiomaticity Detection and Sentence Embedding. Data - https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity


Extracting only the English ones.

In [ ]:
base_path = "/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Data - SemEval 2022/One-Shot"

context_idioms = []
for dirnames, dirs, files in os.walk(base_path):
    for filename in files:
        filename_without_extension, extension = os.path.splitext(filename)
        if extension == '.csv':
            df_new = pd.read_csv(dirnames+"/"+filename)
            # print(dirnames+"/"+filename)

            if 'Label' in df_new.columns:
                if df_new['Label'].isnull().values.any() == False:
                    
                    # Please note that "Idiomatic" is assigned the label 0 in the dataset and 
                    # "non-idiomatic" (including proper nouns) are assigned the label 1
                    # context_idioms.append(df_new.loc[(df_new['Label'] == 0) & (df_new['Language'] == 'EN'), ['MWE', 'Previous', 'Target', 'Next']])
                    context_idioms.append(df_new.loc[df_new['Language'] == 'PT', ['MWE', 'Previous', 'Target', 'Next', 'Label']])

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Data - SemEval 2022/Train/train_zero_shot.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Data - SemEval 2022/Test/test.csv")
df_test.Language.value_counts()

EN    3327
PT    1164
Name: Language, dtype: int64

In [ ]:
df_data_1 = pd.DataFrame(context_idioms[0])
for i in context_idioms[0:]:
    df_data_1 = df_data_1.append(i)

df_data_1

,MWE,Previous,Target,Next,Label
87,segundo plano,"Para tanto, basta ter as versões mais recentes...","A partir daí, a tecnologia do Face ID fica em ...","Para desbloquear o celular usando máscara, o A...",0
88,desfile militar,"Lutaremos até que a junta militar caia”, acres...",Os ataques das forças de segurança ocorreram s...,Atos violentos que afetam a segurança e a esta...,1
89,sangue quente,"Na cobrança, Chiquinho pegou firme e venceu Poli.","Como em todo clássico, o sangue quente apareceu.","Primeiro, Marcel soltou a mão no rosto de Thye...",0
90,sangue quente,Ambos os animais tinham menos de trinta centím...,Os tritilodontídeos em geral eram semelhantes ...,“Cada um dos dois representa a única espécie e...,1
91,alto-falante,Adquiria o Beosound Emerge no site oficial Ban...,"Segundo a empresa, a intenção foi construir um...",O design permite colocar o Beosound Emerge no ...,1
...,...,...,...,...,...
135,amigo oculto,"Mesmo com a pandemia, é possível manter a trad...",Chegou o fim do ano e com ele as festas de Nat...,"Não tem jeito, essa é uma tradição brasileira.",0
136,amigo oculto,Secret Santa Generator : Secret Santa é o nome...,Esta é a versão do app semelhante ao Amigo Ocu...,Eles são análogos e funcionam exatamente da me...,1
137,agente secreto,"Já à meia-noite, o Domingo Maior exibe 3 Dias ...","Ethan Renner (Kevin Costner), veterano agente ...","Seu último desejo é reatar com sua filha, com ...",1
138,sangue frio,"Com a pele em chamas, o companheiro de Khen (P...","Irritado com a mentira, o líder desejará matar...","Assim, ele decidirá se livrar do escolhido e l...",0


In [ ]:
df_data_1 = df_data_1.drop_duplicates()

In [ ]:
df_data_1.shape

(53, 5)

In [ ]:
df_data_1.Label.value_counts()

0    28
1    25
Name: Label, dtype: int64

In [ ]:
df_data_1.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/one-shot-train_data_pt.csv")

### Data 2

Source: https://farkastranslations.com/bilingual_books.php

#### Get the stories first

In [ ]:
# story_path = "/content/Twain_Mark-Tom_Sawyer-en-de-hu-nl-ca.txt"

# f = open(story_path, 'r')
# content = f.read()


In [ ]:
# # html
# story_path_html = "/content/Twain_Mark-Tom_Sawyer-en-de-hu-nl-ca.html"

# table_MN = pd.read_html(story_path_html)
# df = table_MN[0]
# df.shape


# # web html
# story_path_html = "https://farkastranslations.com/books/Twain_Mark-Tom_Sawyer-en-de-hu-nl-ca.html"

# table_MN = pd.read_html(story_path_html)
# df = table_MN[0]
# df.shape

In [ ]:
story_paths = ["https://farkastranslations.com/books/Twain_Mark-Tom_Sawyer-en-de-hu-nl-ca.html",
               "https://farkastranslations.com/books/Austen_Jane-Sense_and_Sensibility-en-es-nl-it.html",
               "https://farkastranslations.com/books/Defoe_Daniel-Robinson_Crusoe-en-hu-es-it-fr-nl.html",
               "https://farkastranslations.com/books/Doyle_Arthur_Conan-Sign_of_Four-en-nl.html",
               "https://farkastranslations.com/books/Dumas_Alexandre-Trois_Mousquetaires-fr-en-hu-es-nl.html",
               "https://farkastranslations.com/books/Twain_Mark-Tom_Sawyer-en-de-hu-nl-ca.html",
               "https://farkastranslations.com/books/Verne_Jules-20000_lieues_sous_les_mers-fr-en-hu-es-nl.html",
               "https://farkastranslations.com/books/Verne_Jules-Tour_du_monde_en_80_jours-fr-en-hu-es-nl.html",
               "https://farkastranslations.com/books/Verne_Jules-Voyage_au_Centre_de_la_Terre-fr-en-es-hu-nl.html",
               "https://farkastranslations.com/books/Carroll_Lewis-Alice_in_wonderland-en-hu-es-it-pt-fr-de-eo-fi.html",
               "https://farkastranslations.com/books/Poe_Edgar_Allan-Fall_of_the_House_of_Usher-en-hu-es-it-fr-de-eo.html",
               "https://farkastranslations.com/books/Carroll_Lewis-Alice_in_wonderland-en-hu-es-it-pt-fr-de-eo-fi.html",
               "https://farkastranslations.com/books/Doyle_Arthur_Conan-Hound_of_the_Baskervilles-en-fr-es-hu-fi-no.html",
               "https://farkastranslations.com/books/France_Anatole-Les_dieux_ont_soif-fr-en-fi.html",
               "https://farkastranslations.com/books/London_Jack_Call_of_the_Wild-en-es-it-fi-sv.html",
               "https://farkastranslations.com/books/Alain_Fournier-Le_grand_Meaulnes-en-fr.html",
               "https://farkastranslations.com/books/Austen_Jane-Pride_and_Prejudice-en-fr-hu.html",
               "https://farkastranslations.com/books/Carroll_Lewis-Alice_in_wonderland-en-hu-es-it-pt-fr-de-eo-fi.html",
               "https://farkastranslations.com/books/Bronte_Charlotte-Jane_Eyre-en-fr-es-it-de-hu.html",
               "https://farkastranslations.com/books/Defoe_Daniel-Moll_Flanders-en-fr.html",
               "https://farkastranslations.com/books/Defoe_Daniel-Robinson_Crusoe-en-hu-es-it-fr-nl.html",
               "https://farkastranslations.com/books/Doyle_Arthur_Conan-Great_Shadow-en-fr.html",
               "https://farkastranslations.com/books/Doyle_Arthur_Conan-Hound_of_the_Baskervilles-en-fr-es-hu-fi-no.html",
               "https://farkastranslations.com/books/Doyle_Arthur_Conan-Rodney_Stone-en-fr.html",
               "https://farkastranslations.com/books/Doyle_Arthur_Conan-A_Study_in_Scarlet-en-fr-es.html",
               "https://farkastranslations.com/books/Dumas_Alexandre-Dame_aux_Camelias-en-fr.html",
               "https://farkastranslations.com/books/Dumas_Alexandre-Trois_Mousquetaires-fr-en-hu-es-nl.html",
               "https://farkastranslations.com/books/Dumas_Alexandre-Trois_Mousquetaires-en-fr.html",
               "https://farkastranslations.com/books/Flaubert_Gustave-Madame_Bovary-fr-en-hu-de.html",
               "https://farkastranslations.com/books/France_Anatole-Les_dieux_ont_soif-fr-en-fi.html",
               "https://farkastranslations.com/books/Hugo_Victor-Notre_Dame_de_Paris-en-fr-de.html",
               "https://farkastranslations.com/books/Jerome_Jerome_K-Three_Men_in_a_Boat-en-fr-it-sv.html",
               "https://farkastranslations.com/books/Maupassant_Guy_de-Pierre_et_Jean-en-fr.html",
               "https://farkastranslations.com/books/Poe_Edgar_Allan-Fall_of_the_House_of_Usher-en-hu-es-it-fr-de-eo.html",
               "https://farkastranslations.com/books/Stendhal-Chartreuse_de_Parme-en-fr.html",
               "https://farkastranslations.com/books/Stendhal-Rouge_et_le_Noir-en-fr.html",
               "https://farkastranslations.com/books/Verne_Jules-20000_lieues_sous_les_mers-fr-en-hu-es-nl.html",
               "https://farkastranslations.com/books/Verne_Jules-Ile_mysterieuse-fr-en-es-hu.html",
               "https://farkastranslations.com/books/Verne_Jules-Tour_du_monde_en_80_jours-en-fr.html",
               "https://farkastranslations.com/books/Verne_Jules-Tour_du_monde_en_80_jours-fr-en-hu-es-nl.html",
               "https://farkastranslations.com/books/Verne_Jules-Voyage_au_Centre_de_la_Terre-fr-en-es-hu-nl.html",
               "https://farkastranslations.com/books/Voltaire-Candide-fr-en-hu-es-el.html",
               "https://farkastranslations.com/books/Wilde_Oscar-Picture_of_Dorian_Gray-en-es-hu-fr-de.html",
               "https://farkastranslations.com/books/Zola_Emile-Germinal-en-fr.html",
               "https://farkastranslations.com/books/Zola_Emile-Therese_Raquin-en-fr.html",]

In [ ]:
story_list = []

# c = 0
for story_path in story_paths:
    table = pd.read_html(story_path)
    df = table[0]
    
    # setting the first row as the column names
    df.columns = df.iloc[0]

    story_list.append(df)


    # if c==5:
    #     break
    # c+=1

In [ ]:
len(story_list)

45

In [ ]:
df_stories = pd.concat(story_list, ignore_index=True)
print(df_stories.shape)
df_stories.head()

(139845, 14)


,English,German,Hungarian,Dutch,Catalan,Spanish,Italian,French,Portuguese,Esperanto,Finnish,Norwegian,Swedish,Greek
0,English,German,Hungarian,Dutch,Catalan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Source: Project Gutenberg,Source: Project Gutenberg,Source: mek.oszk.huTranslation: Koroknay Istvá...,Source: Project Gutenberg,Source: Project GutenbergTranslation: Josep Ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Adventures of Tom Sawyer,Die Abenteuer Tom Sawyers,TOM SAWYER KALANDJAI,De Lotgevallen van Tom Sawyer,Les Aventures De Tom Sawyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mark Twain,Mark Twain,MARK TWAIN,Mark Twain,Mark Twain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PREFACE,Vorwort des Autors.,ELŐSZÓ,NaN,PREFACI.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Get context from the data

In [ ]:
# tokenize at sentence level and store them in a list

from nltk.tokenize import sent_tokenize

tokenized_list = []

# c = 0
for each in df_stories['English']:

    t = sent_tokenize(str(each))
    tokenized_list.append(t)

    # if c == 5:
    #     break
    # c+=1

final_list = [j for sub in tokenized_list for j in sub]

print(len(final_list))
print(final_list[:5])

247581
['English', 'Source: Project Gutenberg', 'The Adventures of Tom Sawyer', 'Mark Twain', 'PREFACE']


In [ ]:
previous_list = []
next_list = []
target_list = []
mwe = []

for ie in idioms_list:

    # look for the sentences with the idioms
    targets = [s for s in final_list if ie in s]

    index = []
    # previous_list = []
    # next_list = []

    for sent in targets:
        index.append(final_list.index(sent))

    # print(index)
    if index:
        for i in index:
            mwe.append(ie)
            target_list.append(final_list[i])
            # print(ie)
            try:
                previous_list.append(final_list[i-1])

            except:
                previous_list.append('___xaina___')

            try:
                next_list.append(final_list[i+1])
            except:
                next_list.append('___xaina___')

df_data_2 = pd.DataFrame(
    {'MWE': mwe,
    'Previous': previous_list,
    'Target': target_list,
    'Next': next_list
    })



In [ ]:
df_data_2

,MWE,Previous,Target,Next
0,white hat,As our tandem came alongside of the four-in-ha...,"My uncle, in his fawn-coloured driving-coat, w...","It was an age of eccentricity, but he had carr..."
1,white hat,"Our mares laid themselves out gallantly, and t...",I found that I could see the black band upon S...,"""We're on the side road to Godstone and Warlin..."
2,white hat,"Round his middle was a canary-yellow sash, and...","He carried a high white hat in his hand, and r...",nan
3,white hat,"Some twenty well-known prize-fighters, includi...",These fellows all wore the high white hats whi...,monogram.
4,white hat,Even with so formidable a guard and such fierc...,In the mean time they formed up in a line of s...,Whilst the business of the fixing of the stake...
...,...,...,...,...
950,ground floor,"Then, after having placed a kettle on the grat...","It was a fairly large room, occupying all the ...",Besides the sideboard of varnished deal the fu...
951,ground floor,I ought to explain to you that the Company ren...,"A large room on the ground floor, two rooms ab...",The decorated gentleman and the lady in the fu...
952,ground floor,M. Hennebeau was standing at the window when H...,"He closed all on the ground floor, and then we...","Unfortunately, it was not possible to shut the..."
953,ground floor,"Étienne freed the old groom, and the soldiers ...","A quarter of an hour later, as the band of str...",The surprise was so great that no one prevente...


In [ ]:
df_data_2.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_2.csv")

### Data in all 3 Languages

In [ ]:
base_path = "/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Data - SemEval 2022"

def get_data(label, lan):
    context_idioms = []
    for dirnames, dirs, files in os.walk(base_path):
        for filename in files:
            filename_without_extension, extension = os.path.splitext(filename)
            if extension == '.csv':
                df_new = pd.read_csv(dirnames+"/"+filename)
                # print(dirnames+"/"+filename)

                if 'Label' in df_new.columns:
                    if df_new['Label'].isnull().values.any() == False:
                        
                        # Please note that "Idiomatic" is assigned the label 0 in the dataset and 
                        # "non-idiomatic" (including proper nouns) are assigned the label 1
                        context_idioms.append(df_new.loc[(df_new['Label'] == label) & (df_new['Language'] == lan), ['MWE', 'Previous', 'Target', 'Next']])

    df_data = pd.DataFrame(context_idioms[0])
    for i in context_idioms[0:]:
        df_data = df_data.append(i)

    df_data = df_data.drop_duplicates()

    return df_data

In [ ]:
# idiomatic label = 0; non-idiomatic label = 1

df_idiomatic_en = get_data(0, 'EN') # idiomatic
df_non_idiomatic_en = get_data(1, 'EN') # non-idiomatic

df_idiomatic_pt = get_data(0, 'PT') # idiomatic
df_non_idiomatic_pt = get_data(1, 'PT') # non-idiomatic

df_idiomatic_gl = get_data(0, 'GL') # idiomatic
df_non_idiomatic_gl = get_data(1, 'GL') # non-idiomatic

In [ ]:
# sizes
print("Idiomatic English:", df_idiomatic_en.shape)
print("Non Idiomatic English:", df_non_idiomatic_en.shape)
print("Idiomatic Portugese:", df_idiomatic_pt.shape)
print("Non Idiomatic Portugese:", df_non_idiomatic_pt.shape)
print("Idiomatic Galician:", df_idiomatic_gl.shape)
print("Non Idiomatic Galician:", df_non_idiomatic_gl.shape)

Idiomatic English: (2009, 4)
Non Idiomatic English: (1944, 4)
Idiomatic Portugese: (997, 4)
Non Idiomatic Portugese: (566, 4)
Idiomatic Galician: (43, 4)
Non Idiomatic Galician: (20, 4)


##Data Stats

In [44]:
df = pd.read_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/train_data_pt.csv")
# df = pd.read_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/test_data_pt.csv")
# df = pd.read_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Test Result Data/GPT3/EN/few-shot-gpt3-davinci-en-t1_0.csv")
# df = pd.read_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Test Result Data/GPT3/PT/few-shot-gpt3-davinci-pt-t1_0.csv")

In [45]:
df[df['Next'].isnull()].index.tolist()

[]

In [46]:
df.iloc[870]

Unnamed: 0                                                 4197
MWE                                              carne vermelha
Previous      Além disso, Gilcelli tem uma hérnia de 20 cent...
Target        Não posso pegar pesos maiores que dois quilos ...
Next          Mantenho minha fé que, com estudos, alguma equ...
Label                                                         0
Name: 870, dtype: object

In [40]:
df.fillna('', inplace=True)

In [47]:
from statistics import mean

def data_stats(df):
    l = list(df.apply(len))
    print("Max lentgh:", max(l))
    # print(df.iloc[l.index(max(l))])
    print("\nSmallest length:", min(l))
    # print(df.iloc[l.index(min(l))])
    print("\nAvg length:", mean(l))

In [48]:
data_stats(df.loc[df['Label']==0, 'Next'])

Max lentgh: 600

Smallest length: 19

Avg length: 135.4968789013733


In [49]:
data_stats(df.loc[df['Label']==1, 'Next'])

Max lentgh: 739

Smallest length: 17

Avg length: 136.12019230769232


In [32]:
1data_stats(df.loc[df['Label']==0, 'Previous'])

Max lentgh: 1231

Smallest length: 4

Avg length: 133.45373467112597


In [33]:
data_stats(df.loc[df['Label']==0, 'Target'])

Max lentgh: 505

Smallest length: 20

Avg length: 151.00334448160535


In [ ]:
data_stats(df.loc[df['Label']==0, 'Next'])

Max lentgh: 394

Smallest length: 5

Avg length: 131.8961038961039


In [ ]:
data_stats(df.loc[df['Label']==1, 'Previous'])

Max lentgh: 376

Smallest length: 37

Avg length: 151.1344537815126


In [ ]:
data_stats(df.loc[df['Label']==1, 'Target'])

Max lentgh: 440

Smallest length: 34

Avg length: 175.21008403361344


In [ ]:
data_stats(df.loc[df['Label']==1, 'Next'])

Max lentgh: 371

Smallest length: 26

Avg length: 136.18487394957984


In [ ]:
# data_stats(df['Generated Next'])

In [ ]:
data_stats(df['Previous'])

Max lentgh: 443

Smallest length: 17

Avg length: 138.5164835164835


In [ ]:
data_stats(df['Target'])

Max lentgh: 512

Smallest length: 32

Avg length: 167.13919413919413


In [ ]:
data_stats(df['Next'])

Max lentgh: 394

Smallest length: 5

Avg length: 133.76556776556777


In [ ]:
# # export the data

# df_idiomatic_en.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_en_idio.csv")
# df_non_idiomatic_en.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_en_non_idio.csv")

# df_idiomatic_pt.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_pt_idio.csv")
# df_non_idiomatic_pt.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_pt_non_idio.csv")

# df_idiomatic_gl.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_gl_idio.csv")
# df_non_idiomatic_gl.to_csv("/content/drive/MyDrive/PSU Stuffs/NLP Lab/M-Idioms/Experiments/Actual Data/data_gl_non_idio.csv")